<a id="top"></a>

# 02 · Data Smoke & Bench

**Qué hace este notebook:**

Pruebas rápidas para validar **datos/splits**, comprobar el **pipeline de codificación temporal** (offline H5 o runtime) y hacer **micro-bench** de *throughput*. Lee la configuración desde `configs/presets.yaml` para asegurar coherencia con el cuaderno 03.

---

## 🎯 Objetivos
- Detectar rápido problemas en datos/H5
- Tener una referencia de it/s y formas de tensores antes de lanzar entrenos largos.

<a id="toc"></a>

## 🧭 Índice

- [1) Setup del entorno e imports](#sec-01)
- [2) Cargar preset y eco de configuración](#sec-02)
- [3) Verificación de datos y carga de `task_list`](#sec-03)
- [4) Factory de DataLoaders (H5 offline o CSV + runtime)](#sec-04)
- [5) Smoke universal (loader → modelo)](#sec-05)
- [6) Forward manual con fallback AMP](#sec-06)
- [7) Echo de configuración del bench](#sec-07)
- [8) Toggle: *it/s* por época](#sec-08)
- [9) (Opc.) Micro-bench pipeline *it/s*](#sec-09)
- [10) (Opc.) Smokes de métodos con `run_continual`](#sec-10)


<a id="sec-01"></a>

## 1) Setup del entorno e imports

**Objetivo:** preparar el entorno para micro-benchmarks y *smoke tests* de los loaders/modelo.

- Configura hilos BLAS (evita over-subscription).
- Detecta `ROOT` y añade el repo al `sys.path`.
- Importa utilidades de *bench*: `make_loader_fn_factory`, `universal_smoke_forward`, etc.
- Selecciona dispositivo (`cuda` si está) y activa optimizaciones (TF32, cuDNN benchmark).

> Este notebook **no entrena**; su objetivo es validar *data → loader → forward* y medir throughput base.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Imports y setup
# =============================================================================
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

from pathlib import Path
import sys, json, torch

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.utils import load_preset, set_seeds, build_make_loader_fn  # ← nuevo
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model
from src.training import set_encode_runtime
import src.training as training
from src.bench import (
    universal_smoke_forward,
    enable_epoch_ips, disable_epoch_ips,
    print_bench_config,
    pipeline_its,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")
print("Device:", device)


<a id="sec-02"></a>

## 2) Config desde `presets.yaml` y eco de configuración

**Objetivo:** usar el mismo preset del proyecto para alinear el bench con entrenamiento.

Incluye:
- Lectura de `PRESET` y extracción de:
  - Modelo + `ImageTransform`.
  - Parámetros de codificación temporal (`ENCODER`, `T`, `GAIN`, `SEED`).
  - Parámetros de DataLoader (workers, prefetch, pin/persistent).
  - *Augment* (`AUG_CFG`) y balanceo online (si aplica).
- Define `make_model_fn(tfm)` para instanciar el modelo con los mismos hiperparámetros neuronales.

> Así, cualquier *smoke/bench* que hagas aquí usa exactamente la misma configuración que 03_TRAIN_CONTINUAL.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Config: lee presets.yaml
# =============================================================================
PRESET = "fast"  # fast | std | accurate
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)

# Modelo / tfm
MODEL_NAME = CFG["model"]["name"]
tfm = ImageTransform(
    CFG["model"]["img_w"], CFG["model"]["img_h"],
    to_gray=bool(CFG["model"]["to_gray"]), crop_top=None
)

# Datos / codificación temporal
ENCODER = CFG["data"]["encoder"]
T       = int(CFG["data"]["T"])
GAIN    = float(CFG["data"]["gain"])
SEED    = int(CFG["data"]["seed"])

USE_OFFLINE_SPIKES   = bool(CFG["data"].get("use_offline_spikes", False))
RUNTIME_ENCODE       = bool(CFG["data"].get("encode_runtime", False))

# ---- DataLoader / augment / balanceo ----------------------------------------
NUM_WORKERS = int(CFG["data"].get("num_workers") or 0)           # ← robusto ante None
PREFETCH    = CFG["data"].get("prefetch_factor")                  # puede ser None
PIN_MEMORY  = bool(CFG["data"].get("pin_memory", True))
PERSISTENT  = bool(CFG["data"].get("persistent_workers", True))

AUG_CFG = AugmentConfig(**(CFG["data"].get("aug_train") or {})) \
          if CFG["data"].get("aug_train") else None

USE_ONLINE_BALANCING = bool(CFG["data"].get("balance_online", False))
BAL_BINS = int(CFG["data"].get("balance_bins") or 50)            # ← robusto ante None
BAL_EPS  = float(CFG["data"].get("balance_smooth_eps") or 1e-3)  # ← robusto ante None

print(f"[PRESET={PRESET}] model={MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] encoder={ENCODER} T={T} gain={GAIN} seed={SEED}")
print(f"[LOADER] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")
print(f"[BALANCE] online={USE_ONLINE_BALANCING} bins={BAL_BINS}")
print(f"[RUNTIME_ENCODE] {RUNTIME_ENCODE} | [OFFLINE_SPIKES] {USE_OFFLINE_SPIKES}")

def make_model_fn(tfm):
    # kwargs solo necesarios para pilotnet_snn
    return build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)


<a id="sec-03"></a>

## 3) Verificación de datos y carga de `task_list`

Comprueba `train/val/test.csv` por recorrido y el `train_balanced.csv` si activas modo offline balanceado. Construye `task_list` desde `tasks.json`/`tasks_balanced.json`.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Verificación de datos + selección de tasks según el PRESET
# =============================================================================
from pathlib import Path as _P
import json

PROC = ROOT / "data" / "processed"
RAW  = ROOT / "data" / "raw" / "udacity"

# --- Elegir tasks según el preset ---
tb_name = CFG["prep"]["tasks_balanced_file_name"]   # p.ej., "tasks_balanced.json"
t_name  = CFG["prep"]["tasks_file_name"]            # p.ej., "tasks.json"
USE_BALANCED = bool(CFG.get("prep_offline", {}).get("use_balanced_tasks", False))

cand_bal = PROC / tb_name
cand_std = PROC / t_name

if USE_BALANCED and cand_bal.exists():
    TASKS_FILE = cand_bal
else:
    TASKS_FILE = cand_std

with open(TASKS_FILE, "r", encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]
print("Usando tasks:", TASKS_FILE.name)
for t in task_list:
    print(f" - {t['name']}: {_P(t['paths']['train']).name}")

# --- Guardarraíl: si usamos tasks balanceadas, aseguramos train_balanced.csv ---
if USE_BALANCED:
    for t in task_list:
        train_path = _P(t["paths"]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' en tasks balanceadas, "
                f"pero encontré '{train_path.name}'."
            )
    print("Verificación de tasks balanceadas OK.")

# --- Verificación específica según el modo de datos ---
if USE_OFFLINE_SPIKES:
    # Comprobamos que existen los H5 requeridos por el preset
    mw, mh = CFG["model"]["img_w"], CFG["model"]["img_h"]
    color = "gray" if CFG["model"]["to_gray"] else "rgb"
    gain_tag = (GAIN if ENCODER == "rate" else 0)
    missing = []

    for t in task_list:
        run = t["name"]
        base = PROC / run
        for split in ("train", "val", "test"):
            expected = base / f"{split}_{ENCODER}_T{T}_gain{gain_tag}_{color}_{mw}x{mh}.h5"
            if not expected.exists():
                missing.append(str(expected))

    if missing:
        raise FileNotFoundError(
            "Faltan H5 requeridos por el preset. Genera primero con 02_ENCODE_OFFLINE.ipynb.\n"
            + "\n".join(" - " + m for m in missing)
        )
    print("OK: H5 requeridos encontrados para todos los splits.")

else:
    # CSV + encode en runtime: comprobamos que el CSV existe y que hay imágenes accesibles
    import pandas as _pd

    for t in task_list:
        for split in ("train", "val", "test"):
            csv_path = _P(tasks_json["splits"][t["name"]][split])
            if not csv_path.is_absolute():
                csv_path = ROOT / csv_path
            if not csv_path.exists():
                raise FileNotFoundError(f"No existe CSV: {csv_path}")

    # Muestra mínima para detectar rutas rotas (ej. 'aug/' sin materializar)
    run0 = task_list[0]["name"]
    csv0 = _P(tasks_json["splits"][run0]["train"])
    if not csv0.is_absolute():
        csv0 = ROOT / csv0
    df0 = _pd.read_csv(csv0, nrows=5)
    col_path = "path" if "path" in df0.columns else None
    if col_path is None:
        # fallback: si el CSV tiene 'center/left/right', tomamos 'center'
        for c in ("center", "image", "img"):
            if c in df0.columns:
                col_path = c
                break
    if col_path is None:
        raise RuntimeError(
            f"No encuentro columna de ruta en {csv0}. Esperaba 'path' o 'center'."
        )

    missing_imgs = [p for p in df0[col_path].tolist() if not _P(p).exists()]
    if missing_imgs:
        raise FileNotFoundError(
            "Rutas de imagen no válidas en CSV (ejemplo):\n"
            f" - {missing_imgs[0]}\n"
            "Si esperabas imágenes balanceadas en 'aug/', re-ejecuta 01A_PREP_BALANCED.ipynb."
        )
    print("OK: CSV y rutas de imágenes accesibles para encode en runtime.")


<a id="sec-04"></a>

## 4) Factory de DataLoaders (H5 offline o CSV + runtime)

Crea `make_loader_fn` con los flags del preset (workers, prefetch, pin, persistent, augment, balanceo online). No pases `SEED` aquí.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Factory de loaders (H5 offline o CSV + runtime encode)
#   Igual que en 03_TRAIN_CONTINUAL: elige H5 si use_offline_spikes=True,
#   y si no hay H5 cae a CSV + encode en GPU.
# =============================================================================
_raw_make_loader_fn = build_make_loader_fn(
    root=ROOT,
    use_offline_spikes=USE_OFFLINE_SPIKES,
    encode_runtime=RUNTIME_ENCODE,
)

# kwargs comunes del DataLoader que queremos aplicar por defecto
_DL_KW = dict(
    num_workers=NUM_WORKERS,
    prefetch_factor=PREFETCH,
    pin_memory=PIN_MEMORY,
    persistent_workers=PERSISTENT,
    aug_train=AUG_CFG,
    balance_train=USE_ONLINE_BALANCING,
    balance_bins=BAL_BINS,
    balance_smooth_eps=BAL_EPS,
)

def make_loader_fn(task, batch_size, encoder, T, gain, tfm, seed, **dl_kwargs):
    """Wrapper pass-through (permite override puntual de kwargs si se pasan)."""
    return _raw_make_loader_fn(
        task=task, batch_size=batch_size, encoder=encoder, T=T, gain=gain,
        tfm=tfm, seed=seed, **{**_DL_KW, **dl_kwargs}
    )

print("make_loader_fn listo (H5 si use_offline_spikes=True; fallback CSV+runtime si no).")


<a id="sec-05"></a>

## 5) Smoke universal (loader → modelo)

Verifica que el primer `task` produce batches válidos y que el modelo hace *forward* sin errores (AMP si hay CUDA). Usa `ENCODER`, `T`, `GAIN` del preset.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Smoke universal de loader+modelo (usa variables del preset)
# =============================================================================
_ = universal_smoke_forward(
    make_loader_fn,
    task=task_list[0],
    encoder=ENCODER, T=T, gain=GAIN,
    tfm=tfm, seed=SEED, device=device,
    use_encode_runtime=RUNTIME_ENCODE,
)


<a id="sec-06"></a>

## 6) Forward manual con fallback AMP

Ejecuta el camino completo manual: loader → (T,B,C,H,W) → modelo, con AMP si está disponible y limpieza del runtime-encode.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Forward manual: loader -> (T,B,C,H,W) con fallback AMP
# =============================================================================
import torch, src.training as training

# 1) Loader pequeño con tu helper
tr, va, te = make_loader_fn(
    task=task_list[0],
    batch_size=8,
    encoder=ENCODER,   # <-- preset
    T=T,               # <-- preset
    gain=GAIN,         # <-- preset
    tfm=tfm,
    seed=SEED,
)

xb, yb = next(iter(tr))
print("batch del loader:", xb.shape, yb.shape)

# 2) A (T,B,C,H,W) según formato de entrada
if xb.ndim == 5:  # (B,T,C,H,W)
    x5d = xb.permute(1,0,2,3,4).contiguous()
    used_runtime_encode = False
    print("dataset ya codificado; solo permuto a (T,B,C,H,W)")
elif xb.ndim == 4:  # (B,C,H,W)
    set_encode_runtime(
        mode=ENCODER, T=T, gain=GAIN,
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    )
    x5d = training._permute_if_needed(xb)  # encode+permuta -> (T,B,C,H,W)
    used_runtime_encode = True
    print("dataset 4D; uso encode en GPU y permuto a (T,B,C,H,W)")
else:
    raise RuntimeError(f"Forma inesperada del batch: {xb.shape}")

print("x5d.device:", x5d.device, "| shape:", tuple(x5d.shape))

# 3) Modelo y forward con fallback AMP
model = make_model_fn(tfm).to(device).eval()

def forward_with_auto_amp(model, x5d, device):
    if torch.cuda.is_available():
        try:
            x_amp = x5d.to(device, dtype=torch.float16, non_blocking=True)
            with torch.inference_mode(), torch.amp.autocast('cuda', enabled=True):
                y = model(x_amp)
            print("[forward] ejecutado con AMP (fp16)")
            return y
        except Exception as e:
            print("[forward] AMP falló, reintento en FP32. Motivo:", str(e))

    x_fp32 = x5d.to(device, dtype=torch.float32, non_blocking=True)
    with torch.inference_mode():
        y = model(x_fp32)
    print("[forward] ejecutado en FP32")
    return y

yhat = forward_with_auto_amp(model, x5d, device)
print("yhat:", tuple(yhat.shape))

# 4) Limpieza del runtime encode (si se usó)
if used_runtime_encode:
    training.set_encode_runtime(None)


<a id="sec-07"></a>

## 7) Echo de configuración del bench

Imprime los parámetros efectivos de DataLoader y balanceo. Útil para rastrear cambios de preset.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Bench: echo de configuración efectiva
# =============================================================================
print_bench_config(
    NUM_WORKERS=NUM_WORKERS, PREFETCH=PREFETCH,
    PIN_MEMORY=PIN_MEMORY, PERSISTENT=PERSISTENT,
    USE_ONLINE_BALANCING=USE_ONLINE_BALANCING,  # ← sin USE_OFFLINE_BALANCED
)


<a id="sec-08"></a>

## 8) Toggle: *it/s* por época

Activa un parche temporal que imprime *iterations/second* por época durante entrenamiento. Llama a `disable_epoch_ips()` para restaurar.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# Toggle: imprimir it/s por época durante entrenamiento
# =============================================================================
enable_epoch_ips()
print("it/s por época ACTIVADO. Llama a disable_epoch_ips() para restaurar.")


<a id="sec-09"></a>

## 9) (Opcional) Micro-bench pipeline *it/s*

Mide *throughput* aproximado iterando `loader → modelo` varias veces, con AMP si procede.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# (Opcional) Micro-bench pipeline: it/s (loader + modelo)
# =============================================================================
# from src.bench import pipeline_its

# pequeño loader
tr, va, te = make_loader_fn(
    task=task_list[0],
    batch_size=8,
    encoder=ENCODER, T=T, gain=GAIN,
    tfm=tfm, seed=SEED,
)

# preparar un batch inicial
xb0, _ = next(iter(tr))
if xb0.ndim == 4:
    set_encode_runtime(mode=ENCODER, T=T, gain=GAIN, device=device)

its = pipeline_its(
    model=make_model_fn(tfm).to(device).eval(),
    loader=tr, device=device,
    iters=100, use_amp=True,
    encoder=ENCODER, T=T, gain=GAIN,
)
print(f"pipeline it/s ≈ {its:.2f}")


<a id="sec-10"></a>

## 10) (Opcional) Smokes de métodos con `run_continual`

Lanza runs cortos por método (`naive`, `ewc`, `rehearsal`, `rehearsal+ewc`) usando la **misma** `CFG` del preset para asegurar coherencia con el 03.

[↑ Volver al índice](#toc)


In [ ]:
# =============================================================================
# (Opcional) Smokes de métodos con run_continual (cfg del preset)
# =============================================================================
from copy import deepcopy
from src.runner import run_continual

DO_RUNS = True  # pon False para saltar

if DO_RUNS:
    base_cfg = load_preset(ROOT / "configs" / "presets.yaml", PRESET)
    base_cfg = deepcopy(base_cfg)
    base_cfg["data"]["seed"] = SEED  # fija la semilla si quieres

    METHODS = {
        "naive": {},
        "ewc": {"lam": 1e9, "fisher_batches": 200},
        "rehearsal": {"buffer_size": 5000, "replay_ratio": 0.2},
        "rehearsal+ewc": {"buffer_size": 5000, "replay_ratio": 0.2, "lam": 7e8, "fisher_batches": 200},
    }

    out_runs = []
    for mname, mparams in METHODS.items():
        cfg_i = deepcopy(base_cfg)
        cfg_i["continual"]["method"] = mname
        cfg_i["continual"]["params"] = mparams

        print(
            f"\n>>> SMOKE: preset={PRESET} | method={mname} | "
            f"seed={cfg_i['data']['seed']} | enc={cfg_i['data']['encoder']} | kwargs={mparams}"
        )
        out_dir, _ = run_continual(
            task_list=task_list,
            make_loader_fn=make_loader_fn,  # factory de Celda 4
            make_model_fn=make_model_fn,
            tfm=tfm,
            cfg=cfg_i,               # CONFIG COMPLETA del preset con el método cambiado
            preset_name=PRESET,      # solo naming
            out_root=ROOT / "outputs",
            verbose=True,
        )
        out_runs.append(out_dir)
    print("\nHecho:", [str(p) for p in out_runs])
